In [1]:
%load_ext autoreload
%autoreload 2

https://www.stata.com/features/overview/logistic-regression/

In [2]:
import numpy as np
import pandas as pd

/Users/fukuitomohiro/.pyenv/versions/3.7.3/envs/quantmetrics/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
import quantmetrics
from quantmetrics.datasets import webuse
from quantmetrics.discrete_choice import logistic

In [10]:
webuse('lbw')

In [4]:
data = pd.read_stata("data/lbw")

In [21]:
X = data.drop('low', axis=1)
y = data['low']
logistic(X, y)

   id  age  lwt  ptl  ht  ui  ftv   bwt  race_black  race_other  smoke_smoker
0  85   19  182    0   0   1    0  2523           1           0             0
1  86   33  155    0   0   0    3  2551           0           1             0
2  87   20  105    0   0   0    1  2557           0           0             1
3  88   21  108    0   0   1    2  2594           0           0             1
4  89   18  107    0   0   1    0  2600           0           0             1
Logistic regression
Number of obs = 189
LR chi2(42.195) = 42.195
Prob > chi2 = 0.42195
Pseudo R2 = 0.42195
Odds Ratio = {'age': 42.195, 'lwt': 42.195}
Std. Err. = {'age': 42.195, 'lwt': 42.195}
z = {'age': 42.195, 'lwt': 42.195}
p_z = {'age': 42.195, 'lwt': 42.195}
95% Conf. Interval bottom = {'age': 42.195, 'lwt': 42.195}
95% Conf. Interval top = {'age': 42.195, 'lwt': 42.195}


In [6]:
data.head()

,id,low,age,lwt,race,smoke,ptl,ht,ui,ftv,bwt
0,85,0,19,182,black,nonsmoker,0,0,1,0,2523
1,86,0,33,155,other,nonsmoker,0,0,0,3,2551
2,87,0,20,105,white,smoker,0,0,0,1,2557
3,88,0,21,108,white,smoker,0,0,1,2,2594
4,89,0,18,107,white,smoker,0,0,1,0,2600


方針としては、ダミー変数を置き換える。(多重共線性に気をつける)(一番上のidのデータを保持しておくことで、ダミー化されていない記号を頭に入れておく)  
尤度関数の最適化をscipyの関数を使って行い、結果を出力  
引数として、group(stataのi.と同じでダミー変数を作る項を指定、defaultだと数値はそのまま扱い、文字列はダミー変数処理とかでいいと思う)  
ダミー変数で基準となる項目の指定とかも引数にできたらいいかも

In [9]:
data['race'].value_counts()

white    96
other    67
black    26
Name: race, dtype: int64

In [10]:
data['smoke'].value_counts()

nonsmoker    115
smoker        74
Name: smoke, dtype: int64

おそらく一番数が多いやつがdefaultになっている。

In [11]:
data['race'].value_counts().index[0]

'white'

In [12]:
data['smoke'].value_counts().index[0]

'nonsmoker'

In [13]:
data.dtypes

id          int16
low          int8
age          int8
lwt         int16
race     category
smoke    category
ptl          int8
ht           int8
ui           int8
ftv          int8
bwt         int16
dtype: object

In [14]:
category_variables = data.dtypes[(data.dtypes == 'category') | (data.dtypes == 'object')].index

In [15]:
data.dtypes == 'category'

id       False
low      False
age      False
lwt      False
race      True
smoke     True
ptl      False
ht       False
ui       False
ftv      False
bwt      False
dtype: bool

In [16]:
category_variables = data.dtypes[(data.dtypes == 'category') | (data.dtypes == 'object')].index
category_mode = {}
for variable in category_variables:
    category_mode[variable] = data[variable].value_counts().index[0]
print(category_mode)

{'race': 'white', 'smoke': 'nonsmoker'}


In [7]:
pd.get_dummies(data)

,id,low,age,lwt,ptl,ht,ui,ftv,bwt,race_white,race_black,race_other,smoke_nonsmoker,smoke_smoker
0,85,0,19,182,0,0,1,0,2523,0,1,0,1,0
1,86,0,33,155,0,0,0,3,2551,0,0,1,1,0
2,87,0,20,105,0,0,0,1,2557,1,0,0,0,1
3,88,0,21,108,0,0,1,2,2594,1,0,0,0,1
4,89,0,18,107,0,0,1,0,2600,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,79,1,28,95,0,0,0,2,2466,1,0,0,0,1
185,81,1,14,100,0,0,0,2,2495,0,0,1,1,0
186,82,1,23,94,0,0,0,0,2495,0,0,1,0,1
187,83,1,17,142,0,1,0,0,2495,0,1,0,1,0


In [19]:
for i, j in category_mode.items():
    print(pd.get_dummies(data)[i + '_' + j])

0      0
1      0
2      1
3      1
4      1
      ..
184    1
185    0
186    0
187    0
188    1
Name: race_white, Length: 189, dtype: uint8
0      1
1      1
2      0
3      0
4      0
      ..
184    0
185    1
186    0
187    1
188    0
Name: smoke_nonsmoker, Length: 189, dtype: uint8
